In [204]:
import pandas as pd
import csv
import numpy as np
import os

# Dictionaries

In [205]:
# Change the values that need to be changed manually here
station_coord_rects = {
    'Xavier Hall': { 
        'Latitude': { 
            'lower': 14.640004,
            'higher': 14.640371
        },
        'Longitude': { 
            'lower': 121.078251,
            'higher': 121.078789
        }
    },
    'Fine Arts Annex': { 
        'Latitude': { 
            'lower': 14.636606,
            'higher': 14.636749
        },
        'Longitude': { 
            'lower': 121.078116,
            'higher': 121.078324
        }
    },
    'Loyola House of Studies': { 
        'Latitude': { 
            'lower': 14.636115,
            'higher': 14.636302
        },
        'Longitude': { 
            'lower': 121.080644,
            'higher': 121.080738
        }
    },
    'Grade School': { 
        'Latitude': { 
            'lower': 14.634715,
            'higher': 14.634715
        },
        'Longitude': { 
            'lower': 121.076171,
            'higher': 121.076326
        }
    },
    'JSEC': { 
        'Latitude': { 
            'lower': 14.637686,
            'higher': 14.637804
        },
        'Longitude': { 
            'lower': 121.076260,
            'higher': 121.076370
        }
    },
    'Gate 2.5': { 
        'Latitude': { 
            'lower': 14.637874,
            'higher': 14.637927
        },
        'Longitude': { 
            'lower': 121.074848,
            'higher': 121.075020
        }
    },
    'Leong Hall': { 
        'Latitude': { 
            'lower': 14.640724,
            'higher': 14.640864
        },
        'Longitude': { 
            'lower': 121.076177,
            'higher': 121.076395
        }
    },
    'Power Station': { 
        'Latitude': { 
            'lower': 14.635504,
            'higher': 14.635692
        },
        'Longitude': { 
            'lower': 121.074559,
            'higher': 121.074674
        }
    },
}

In [206]:
next_station_dict = { 
    'Xavier Hall': 'Fine Arts Annex',
    'Fine Arts Annex': 'Loyola House of Studies',
    'Loyola House of Studies': 'Grade School',
    'Grade School': 'Gate 2.5',
    'Gate 2.5': 'Leong Hall',
    'Leong Hall': 'Xavier Hall'
}

# Functions

In [207]:
from datetime import time, datetime

def time_to_daytimestamp(stamp: str) -> int:
    # Handling None or float inputs
    if stamp is None or isinstance(stamp, float):
        return -1
    
    # If stamp is a datetime.datetime object
    if isinstance(stamp, datetime):
        year_seconds = stamp.year * 365 * 24 * 3600  # Assuming average year length
        month_seconds = stamp.month * 30 * 24 * 3600  # Assuming average month length
        day_seconds = stamp.day * 24 * 3600
        return year_seconds + month_seconds + day_seconds + stamp.hour * 3600 + stamp.minute * 60 + stamp.second
    
    # If stamp is a datetime.time object, you can directly access hours, minutes, and seconds
    if isinstance(stamp, time):
        return stamp.hour * 3600 + stamp.minute * 60 + stamp.second

    # If stamp is a string in "HH:MM:SS" format
    h, m, s = stamp.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

In [208]:
# Reformat Lat and Long to be decimal coordinates
def ddmm_mmmm_to_decimal(coord):
    # Assuming coord is in the format 'ddmm.mmmm'
    
    # Extract degrees and decimal minutes
    degrees = coord // 100
    decimal_minutes = coord % 100
    
    # Convert decimal minutes to decimal degrees
    decimal_degrees = degrees + decimal_minutes / 60
    
    return decimal_degrees

In [209]:
# Label stations along the line that the ejeep is in based on coordinates
def coord_to_station(lat, long) -> str:
    stations = station_coord_rects.keys()

    for station in stations:
        lat_bounds, long_bounds = station_coord_rects[station]['Latitude'], station_coord_rects[station]['Longitude']

        if (lat >= lat_bounds['lower'] - 0.0003 and lat <= lat_bounds['higher'] + 0.0003) and (long >= long_bounds['lower'] - 0.0003 and long <= long_bounds['higher'] + 0.0003):
            return station
    else:
        return "En Route"

In [210]:
import math

# Function to calculate distance using Haversine formula
def haversine_distance(lat_diff, lon_diff):
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude differences from degrees to radians
    lat_diff_rad = math.radians(lat_diff)
    lon_diff_rad = math.radians(lon_diff)

    # Haversine formula
    a = math.sin(lat_diff_rad / 2)**2 + math.cos(0) * math.cos(0) * math.sin(lon_diff_rad / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Calculate distance
    distance = R * c

    return distance * 1000

In [211]:
# Make dictionary of all stations & average lat & long
def station_centroids(coord) -> dict:
    new_dict = dict()

    new_dict['En Route'] = None

    for station in station_coord_rects:
        ave = (station_coord_rects[station][coord]['lower'] + station_coord_rects[station][coord]['higher']) / 2

        new_dict[station] = ave
    
    return new_dict

lat_centers = station_centroids('Latitude')
long_centers = station_centroids('Longitude')

lat_centers
long_centers

{'En Route': None,
 'Xavier Hall': 121.07852,
 'Fine Arts Annex': 121.07821999999999,
 'Loyola House of Studies': 121.080691,
 'Grade School': 121.07624849999999,
 'JSEC': 121.076315,
 'Gate 2.5': 121.074934,
 'Leong Hall': 121.07628600000001,
 'Power Station': 121.07461649999999}

In [212]:
# Define Prev Station
def prev_station(df):
    df['Prev Station'] = df['Station']
    df.loc[df['Station'] == 'En Route', "Prev Station"] = np.nan

    df['Prev Station'] = df['Prev Station'].fillna(method='ffill')
    df.loc[df['Station'] != 'En Route', "Prev Station"] = np.nan
    df['Prev Station'] = df['Prev Station'].fillna(method='ffill')
    df['Prev Station'] = df['Prev Station'].fillna(method='bfill')
    
    return df

In [213]:
# Define Next Station
def next_station(df):
    df['Next Station'] = df['Station']
    df.loc[df['Station'] == 'En Route', "Next Station"] = np.nan

    df['Next Station'] = df['Next Station'].fillna(method='bfill')
    df.loc[df['Station'] != 'En Route', "Next Station"] = np.nan
    df['Next Station'] = df['Next Station'].fillna(method='bfill')
    df['Next Station'] = df['Next Station'].fillna(method='ffill')
    
    return df

In [214]:
# Previous & Next Times

def prev_next_times(df):
    # Create a 2nd df that's a more compiled version of the table for easier look-ups

    df2 = pd.DataFrame(columns=['Original Index', 'Station', 'Time'])

    next_station = ''

    for index, row in df.iterrows():
        if row['Station'] == 'En Route' or row['Station'] == 'Power Station':
            continue

        # Log the first instance of the ejeep getting to the stop
        if row['Station'] == next_station or next_station == '':
            new_row = {
                'Original Index': index,
                'Station': row['Station'],
                'Time': row['Time']
            }
            df2.loc[len(df2)] = new_row

            next_station = next_station_dict[row['Station']]
            
    # This essentially notes when the e-jeep arrives and leaves at the station

    df_stations_sum = pd.DataFrame(columns=['Original Index', 'Station', 'Time', 'Status'])

    df_stations_sum = pd.DataFrame(columns=['Original Index', 'Station', 'Time', 'Status'])

    previous_index = None
    previous_station = 'En Route'
    previous_time = None

    for index, row in df.iterrows():
        current_time, current_station = row['Time'], row['Station']

        if previous_station == 'En Route' and current_station == 'En Route':
            continue

        if previous_station == 'En Route' and current_station != 'En Route':
            # Arrived at the station
            new_row = { 
                'Original Index': index,
                'Station': current_station,
                'Time': current_time,
                'Status': 'Arriving'
            }

            df_stations_sum.loc[len(df_stations_sum)] = new_row
        
        if previous_station != 'En Route' and current_station == 'En Route':
            # Departed from the station
            new_row = { 
                'Original Index': previous_index,
                'Station': previous_station,
                'Time': previous_time,
                'Status': 'Departing'
            }

            df_stations_sum.loc[len(df_stations_sum)] = new_row
        
        previous_time, previous_index, previous_station = current_time, index, current_station
    
    df['Previous Time to Xavier Hall'] = None
    df['Previous Time to Fine Arts Annex'] = None
    df['Previous Time to Loyola House of Studies'] = None
    df['Previous Time to Grade School'] = None
    df['Previous Time to Gate 2.5'] = None
    df['Previous Time to Leong Hall'] = None

    df['Next Time to Xavier Hall'] = None
    df['Next Time to Fine Arts Annex'] = None
    df['Next Time to Loyola House of Studies'] = None
    df['Next Time to Grade School'] = None
    df['Next Time to Gate 2.5'] = None
    df['Next Time to Leong Hall'] = None
    
    df2 = df_stations_sum.copy()

    for index, row in df2.iterrows():
        future_df = df2.iloc[index + 1: index + 40]
        prev_df = df2.iloc[max(index - 40, 0): max(0, index - 1)]

        if row['Status'] == 'Departing':
            for station in next_station_dict.keys():
                future_df_index = future_df.index.where(future_df['Station'] == station).min()
                if not pd.isnull(future_df_index):
                    future_df_index = int(future_df_index)
                    df.at[row['Original Index'], f'Next Time to {station}'] = df2.at[future_df_index, 'Time']
        elif row['Status'] == 'Arriving':
            for station in next_station_dict.keys():
                prev_df_index = prev_df.index.where(prev_df['Station'] == station).max()
                if not pd.isnull(prev_df_index):
                    prev_df_index = int(prev_df_index)
                    df.at[row['Original Index'], f'Previous Time to {station}'] = df2.at[prev_df_index, 'Time']
                    
    df = df.dropna(how='all')
    
    for station in next_station_dict.keys():
        df[f'Previous Time to {station}'] = df[f'Previous Time to {station}'].interpolate(method='ffill', limit_direction='forward')
        df[f'Next Time to {station}'] = df[f'Next Time to {station}'].interpolate(method='bfill', limit_direction='backward')
    
    return df

In [215]:

def process_file(file_path):
     df = pd.read_csv(file_path)
     df.tail(10)

     df.columns = ['Type', 'Datetime', 'Lat', 'Long', 'Altitude', 'Humidity', 'Temperature']
     df.drop(columns='Type', inplace=True)

     # Drop rows with null at start and end
     # df = df.loc[df.notnull().all(axis=1).argmax():]

     # ids = df[df.notnull().all(axis=1)].index
     # last_id = ids[-1]

     # df = df.loc[:last_id]
            
     # Try parsing with both formats and handle errors
     def parse_time(time_str):
          try:
               return pd.to_datetime(time_str, format='%H:%M:%S%z')
          except ValueError:
               return pd.to_datetime(time_str, format='%H:%M:%S.%f%z')
            
     # Fix time to PHT
     log_date = os.path.basename(file_path).split('/')[-1]
     # df['Datetime'] = pd.to_datetime(df['Datetime'], format='%H:%M:%S.%f%z')
     df['Datetime'] = df['Datetime'].apply(parse_time)
     df['Datetime'] = df['Datetime'].dt.tz_convert('+08:00')
     df['Datetime'] = pd.to_datetime(df['Datetime'].dt.strftime(f"{log_date[5:15]} %H:%M:%S"))
     
     # Create a DataFrame with a time range for every second between 6am and 6pm
     time_range = pd.date_range(log_date[5:15] + " 06:00:00", log_date[5:15] + " 18:00:00", freq='S')
     expected_df = pd.DataFrame({'Datetime': time_range})
     expected_df['Datetime'] = pd.to_datetime(expected_df['Datetime'])

     # Merge the expected DataFrame with your original DataFrame on the 'time' column
     df = pd.merge(expected_df, df, on='Datetime', how='left')
     
     # Extract Time
     df['Time'] = df['Datetime'].dt.time
     
     # Extract Day
     df['Day'] = df['Datetime'].dt.day

     # Add day of week
     df['Day_of_Week'] = df['Datetime'].dt.day_name()
            
     # Add hour of day
     df['Hour_of_Day'] = df['Datetime'].dt.hour
     
     # Filter out rows before 6am and after 6pm
     df = df.loc[(df['Hour_of_Day'] >= 6) & (df['Hour_of_Day'] <= 18)]

     # Get One Hot Encoded E-Jeep ID
     ejeep_id = file_path.split('/')[1]
     df['IsEJeep1'] = 0
     df['IsEJeep2'] = 0
     df[f"IsEJeep{ejeep_id[5]}"] = 1

     # Iterpolate values for NaN GPS values
     df['Lat'] = df['Lat'].astype(float).interpolate(method='linear', limit_direction='both') 
     df['Long'] = df['Long'].astype(float).interpolate(method='linear', limit_direction='both') 
     df['Altitude'] = df['Altitude'].astype(float).interpolate(method='linear', limit_direction='both') 
     df['Temperature'] = df['Temperature'].astype(float).interpolate(method='linear', limit_direction='both') 
     df['Humidity'] = df['Humidity'].astype(float).interpolate(method='linear', limit_direction='both') 
            
     df['Latitude'] = df.apply(lambda row: ddmm_mmmm_to_decimal(row['Lat']), axis=1)
     df['Longitude'] = df.apply(lambda row: ddmm_mmmm_to_decimal(row['Long']), axis=1)

     # Drop old Lat / Long values
     df.drop(columns=['Lat', 'Long'], inplace=True)
            
     df['Station'] = df.apply(lambda row: coord_to_station(row['Latitude'], row['Longitude']), axis=1)
            
     # Calculate speed (noted as distance since it's by second anyway) based on change in coordinates from the previous
     # Helps establish if ejeep is truly within station

     df['Lat Diff'] = df['Latitude'].diff()
     df['Long Diff'] = df['Longitude'].diff()

     df['Distance'] = df.apply(lambda row: haversine_distance(row['Lat Diff'], row['Long Diff']), axis=1)
     
     df['Lat Diff'] = df['Lat Diff'].fillna(0)
     df['Long Diff'] = df['Long Diff'].fillna(0)
     df['Distance'] = df['Distance'].fillna(0)
     
     df['Station'] = df.apply(lambda row: 'En Route' if row['Station'] != 'En Route' and row['Distance'] >= 0.6 else row['Station'], axis=1)
            
     # Add station column that's true if in station, false if en route
     df["IsStation"] = 0
     df.loc[(df["Station"] != "En Route") & (df["Station"] != "Power Station"), "IsStation"] = 1
     
     # Drop rows before first and last station of each day
     # id_not_null = df.notnull().all(axis=1).idxmax()
     # start = df.loc[df.IsStation==1,'IsStation'].index[0] - id_not_null
     # end = df.query('IsStation == 1').index.max()+1
     # df = df.iloc[start:end]
     
     df = prev_station(df)
     df = next_station(df)
            
     # Filter out repeated stations
     df['Station'] = df.apply(lambda x: x['Next Station'] if x['Station'] == 'En Route' and x['Prev Station'] == x['Next Station'] else x['Station'], axis=1)
     df = prev_station(df)
     df = next_station(df)
            
     df = prev_next_times(df)
     print("done with " + log_date)
     return df

In [216]:
df = process_file("E-Jeep Data/ejeep1/data_2023-11-14_1.csv")
df

done with data_2023-11-14_1.csv


,Datetime,Altitude,Humidity,Temperature,Time,Day,Day_of_Week,Hour_of_Day,IsEJeep1,IsEJeep2,...,Previous Time to Loyola House of Studies,Previous Time to Grade School,Previous Time to Gate 2.5,Previous Time to Leong Hall,Next Time to Xavier Hall,Next Time to Fine Arts Annex,Next Time to Loyola House of Studies,Next Time to Grade School,Next Time to Gate 2.5,Next Time to Leong Hall
0,2023-11-14 06:00:00,61.3,87.900002,27.799999,06:00:00,14,Tuesday,6,1,0,...,None,None,None,None,06:42:04,06:23:39,06:24:38,06:31:31,06:35:15,06:40:02
1,2023-11-14 06:00:01,61.3,87.900002,27.799999,06:00:01,14,Tuesday,6,1,0,...,None,None,None,None,06:42:04,06:23:39,06:24:38,06:31:31,06:35:15,06:40:02
2,2023-11-14 06:00:02,61.3,87.900002,27.799999,06:00:02,14,Tuesday,6,1,0,...,None,None,None,None,06:42:04,06:23:39,06:24:38,06:31:31,06:35:15,06:40:02
3,2023-11-14 06:00:03,61.3,87.900002,27.799999,06:00:03,14,Tuesday,6,1,0,...,None,None,None,None,06:42:04,06:23:39,06:24:38,06:31:31,06:35:15,06:40:02
4,2023-11-14 06:00:04,61.3,87.900002,27.799999,06:00:04,14,Tuesday,6,1,0,...,None,None,None,None,06:42:04,06:23:39,06:24:38,06:31:31,06:35:15,06:40:02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43196,2023-11-14 17:59:56,21.4,65.699997,31.700001,17:59:56,14,Tuesday,17,1,0,...,17:51:07,17:20:24,17:29:54,17:45:11,None,None,None,None,None,None
43197,2023-11-14 17:59:57,21.3,65.699997,31.700001,17:59:57,14,Tuesday,17,1,0,...,17:51:07,17:20:24,17:29:54,17:45:11,None,None,None,None,None,None
43198,2023-11-14 17:59:58,21.2,65.699997,31.700001,17:59:58,14,Tuesday,17,1,0,...,17:51:07,17:20:24,17:29:54,17:45:11,None,None,None,None,None,None
43199,2023-11-14 17:59:59,21.1,65.699997,31.700001,17:59:59,14,Tuesday,17,1,0,...,17:51:07,17:20:24,17:29:54,17:45:11,None,None,None,None,None,None


In [ ]:
df.isna().sum()

# Raw Data
* Combine csv files from each rpi into one data frame
* Pre-process raw data

In [217]:
# Create a list to hold the dataframes
df_list = []

for i in range(2):
    i = i + 1
    folder_path = os.path.join('E-Jeep Data', f"ejeep{i}")
    all_files = os.listdir(folder_path)
    
    # Filter out non-CSV files
    csv_files = [f for f in all_files if f.endswith('.csv')]
    
    for csv in csv_files:
        file_path = os.path.join(folder_path, csv)
        try:
            # Try reading the file using default UTF-8 encoding
            df = process_file(file_path)
            df_list.append(df)
        except UnicodeDecodeError:
            try:
                # If UTF-8 fails, try reading the file using UTF-16 encoding with tab separator
                df = pd.read_csv(file_path, sep='\t', encoding='utf-16')
                df_list.append(df)
            except Exception as e:
                print(f"Could not read file {csv} because of error: {e}")
        except Exception as e:
            print(f"Could not read file {csv} because of error: {e}")

done with data_2023-11-17_1.csv
done with data_2023-11-28_1.csv
done with data_2023-11-15_1.csv
done with data_2023-11-11_1.csv
done with data_2023-11-30_1.csv
done with data_2023-11-14_1.csv
done with data_2023-11-29_1.csv
done with data_2023-11-16_1.csv
done with data_2023-11-10_1.csv
done with data_2023-11-09_1.csv
done with data_2023-11-28_2.csv
done with data_2023-11-15_2.csv
done with data_2023-11-17_2.csv
done with data_2023-11-11_2.csv
done with data_2023-11-16_2.csv
done with data_2023-11-30_2.csv
done with data_2023-11-29_2.csv


In [218]:
# Concatenate all data into one DataFrame
df = pd.concat(df_list, ignore_index=True)
df

,Datetime,Altitude,Humidity,Temperature,Time,Day,Day_of_Week,Hour_of_Day,IsEJeep1,IsEJeep2,...,Previous Time to Loyola House of Studies,Previous Time to Grade School,Previous Time to Gate 2.5,Previous Time to Leong Hall,Next Time to Xavier Hall,Next Time to Fine Arts Annex,Next Time to Loyola House of Studies,Next Time to Grade School,Next Time to Gate 2.5,Next Time to Leong Hall
0,2023-11-17 06:00:00,63.7,79.199997,29.299999,06:00:00,17,Friday,6,1,0,...,None,None,None,None,06:18:02,06:48:05,06:21:45,06:06:27,06:55:48,06:12:32
1,2023-11-17 06:00:01,63.5,79.199997,29.299999,06:00:01,17,Friday,6,1,0,...,None,None,None,None,06:18:02,06:48:05,06:21:45,06:06:27,06:55:48,06:12:32
2,2023-11-17 06:00:02,63.3,79.199997,29.299999,06:00:02,17,Friday,6,1,0,...,None,None,None,None,06:18:02,06:48:05,06:21:45,06:06:27,06:55:48,06:12:32
3,2023-11-17 06:00:03,63.0,79.199997,29.299999,06:00:03,17,Friday,6,1,0,...,None,None,None,None,06:18:02,06:48:05,06:21:45,06:06:27,06:55:48,06:12:32
4,2023-11-17 06:00:04,62.7,79.199997,29.299999,06:00:04,17,Friday,6,1,0,...,None,None,None,None,06:18:02,06:48:05,06:21:45,06:06:27,06:55:48,06:12:32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
734845,2023-11-29 17:59:56,97.3,61.700001,32.599998,17:59:56,29,Wednesday,17,0,1,...,17:39:54,17:44:11,17:17:47,17:35:03,None,None,None,None,None,None
734846,2023-11-29 17:59:57,96.9,61.700001,32.599998,17:59:57,29,Wednesday,17,0,1,...,17:39:54,17:44:11,17:17:47,17:35:03,None,None,None,None,None,None
734847,2023-11-29 17:59:58,96.6,61.700001,32.599998,17:59:58,29,Wednesday,17,0,1,...,17:39:54,17:44:11,17:17:47,17:35:03,None,None,None,None,None,None
734848,2023-11-29 17:59:59,96.0,61.700001,32.599998,17:59:59,29,Wednesday,17,0,1,...,17:39:54,17:44:11,17:17:47,17:35:03,None,None,None,None,None,None


In [219]:
# Sort by datetime
df = df.sort_values(by=['Day', 'IsEJeep2', 'Time'], ascending=True)

# Data Processing

In [220]:
# Add IsCharging column: true if charging, false otherwise
df['IsCharging'] = 0

df.loc[df['Station'] == 'Power Station', "IsCharging"] = 1

In [221]:
# Add distances (by road) to next station
df['Cum Distance'] = df[::-1].groupby((df['Station'] != 'En Route').cumsum())['Distance'].cumsum()
df.loc[df['Station'] != 'En Route', 'Cum Distance'] = 0

df.dtypes

Datetime                                    datetime64[ns]
Altitude                                           float64
Humidity                                           float64
Temperature                                        float64
Time                                                object
Day                                                  int64
Day_of_Week                                         object
Hour_of_Day                                          int64
IsEJeep1                                             int64
IsEJeep2                                             int64
Latitude                                           float64
Longitude                                          float64
Station                                             object
Lat Diff                                           float64
Long Diff                                          float64
Distance                                           float64
IsStation                                            int

In [222]:
# Add distances (as the bird flies) to next station
df['Next Station Lat'] = df['Next Station'].map(lat_centers)
df['Next Station Long'] = df['Next Station'].map(long_centers)

df['Abs Distance'] = df.apply(lambda row: haversine_distance(row['Next Station Lat'] - row['Latitude'], row['Next Station Long'] - row['Longitude']), axis=1)

In [223]:
# Do percentage of distance remaining to next station

# Calculate the maximum value of C within each group of 'X'
max_distance_per_group = df[df['Station'] == 'En Route'].groupby((df['Station'] != 'En Route').cumsum())['Cum Distance'].transform('max')

# Add column D representing the percentage of each value in column C relative to the maximum value within its group
df['Percent Distance'] = df['Cum Distance'] / max_distance_per_group * 100

df.dtypes


Datetime                                    datetime64[ns]
Altitude                                           float64
Humidity                                           float64
Temperature                                        float64
Time                                                object
Day                                                  int64
Day_of_Week                                         object
Hour_of_Day                                          int64
IsEJeep1                                             int64
IsEJeep2                                             int64
Latitude                                           float64
Longitude                                          float64
Station                                             object
Lat Diff                                           float64
Long Diff                                          float64
Distance                                           float64
IsStation                                            int

In [224]:
# Add distance (as the crow flies) to the previous station
df['Prev Station Lat'] = df['Prev Station'].map(lat_centers)
df['Prev Station Long'] = df['Prev Station'].map(long_centers)

df['Prev Abs Distance'] = df.apply(lambda row: haversine_distance(row['Prev Station Lat'] - row['Latitude'], row['Prev Station Long'] - row['Longitude']), axis=1)

In [225]:
# Add distance (by road) to the previous station
df['Prev Cum Distance'] = df.groupby((df['Station'] != 'En Route').cumsum())['Distance'].cumsum()
df.loc[df['Station'] != 'En Route', 'Prev Cum Distance'] = 0

# Ordinal Encoding

In [226]:
df['Prev Station'] = df['Prev Station'].replace(np.nan, 'Power Station')
df['Next Station'] = df['Next Station'].replace(np.nan, 'Power Station')

In [227]:
route = ['En Route', 'Grade School', 'JSEC', 'Gate 2.5', 'Leong Hall', 'Xavier Hall', 'University Dormitory', 'Fine Arts Annex', 'Loyola House of Studies', 'Gate 1', 'Power Station']
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Station
df['Encoded Station'] = df['Station'].apply(lambda x: route.index(x))

# Prev Station
df['Encoded Prev Station'] = df['Prev Station'].apply(lambda x: route.index(x))

# Next Station
df['Encoded Next Station'] = df['Next Station'].apply(lambda x: route.index(x))

# Day of Week
df['Encoded Day'] = df['Day_of_Week'].apply(lambda x: days.index(x))

In [228]:
df.dtypes

Datetime                                    datetime64[ns]
Altitude                                           float64
Humidity                                           float64
Temperature                                        float64
Time                                                object
Day                                                  int64
Day_of_Week                                         object
Hour_of_Day                                          int64
IsEJeep1                                             int64
IsEJeep2                                             int64
Latitude                                           float64
Longitude                                          float64
Station                                             object
Lat Diff                                           float64
Long Diff                                          float64
Distance                                           float64
IsStation                                            int

In [229]:
df.drop(columns='Day_of_Week', inplace=True)
df.drop(columns='Station', inplace=True)
df.drop(columns='Prev Station', inplace=True)
df.drop(columns='Next Station', inplace=True)

In [230]:
# Rearrange columns
order = ['Datetime', 'Day', 'Encoded Day', 'Time', 'Hour_of_Day', 'IsEJeep1', 'IsEJeep2', 'Altitude', 'Humidity', 
         'Temperature', 'Latitude', 'Longitude', 'Lat Diff', 'Long Diff', 'Distance',  'Cum Distance', 'Abs Distance', 'Percent Distance', 
         'Encoded Station', 'IsStation', 'IsCharging', 
         'Encoded Prev Station', 'Prev Station Lat', 'Prev Station Long', 'Prev Abs Distance', 'Prev Cum Distance', 
         'Previous Time to Xavier Hall', 'Previous Time to Fine Arts Annex', 'Previous Time to Loyola House of Studies',
         'Previous Time to Grade School', 'Previous Time to Gate 2.5', 'Previous Time to Leong Hall', 
         'Encoded Next Station', 'Next Station Lat', 'Next Station Long', 'Next Time to Xavier Hall', 'Next Time to Fine Arts Annex', 
         'Next Time to Loyola House of Studies', 'Next Time to Grade School', 'Next Time to Gate 2.5', 'Next Time to Leong Hall', 
        ]
remaining_columns = [col for col in df.columns if col not in order]
# This makes sures that even if a column was not specified, it will still be appended at the end.

df = df[order + remaining_columns]

## TODO: label y with prefix for easy identification
df

,Datetime,Day,Encoded Day,Time,Hour_of_Day,IsEJeep1,IsEJeep2,Altitude,Humidity,Temperature,...,Previous Time to Leong Hall,Encoded Next Station,Next Station Lat,Next Station Long,Next Time to Xavier Hall,Next Time to Fine Arts Annex,Next Time to Loyola House of Studies,Next Time to Grade School,Next Time to Gate 2.5,Next Time to Leong Hall
388827,2023-11-09 06:00:00,9,3,06:00:00,6,1,0,50.3,77.699997,29.500000,...,None,1,14.634715,121.076248,06:23:40,06:48:09,06:26:54,06:16:38,06:35:51,06:20:49
388828,2023-11-09 06:00:01,9,3,06:00:01,6,1,0,50.1,77.699997,29.500000,...,None,1,14.634715,121.076248,06:23:40,06:48:09,06:26:54,06:16:38,06:35:51,06:20:49
388829,2023-11-09 06:00:02,9,3,06:00:02,6,1,0,50.1,77.699997,29.500000,...,None,1,14.634715,121.076248,06:23:40,06:48:09,06:26:54,06:16:38,06:35:51,06:20:49
388830,2023-11-09 06:00:03,9,3,06:00:03,6,1,0,49.8,77.699997,29.500000,...,None,1,14.634715,121.076248,06:23:40,06:48:09,06:26:54,06:16:38,06:35:51,06:20:49
388831,2023-11-09 06:00:04,9,3,06:00:04,6,1,0,49.8,77.699997,29.500000,...,None,1,14.634715,121.076248,06:23:40,06:48:09,06:26:54,06:16:38,06:35:51,06:20:49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691640,2023-11-30 17:59:56,30,3,17:59:56,17,0,1,53.1,59.400002,32.700001,...,17:52:14,8,14.636209,121.080691,None,None,None,None,None,None
691641,2023-11-30 17:59:57,30,3,17:59:57,17,0,1,52.6,59.400002,32.700001,...,17:52:14,8,14.636209,121.080691,None,None,None,None,None,None
691642,2023-11-30 17:59:58,30,3,17:59:58,17,0,1,51.8,59.400002,32.700001,...,17:52:14,8,14.636209,121.080691,None,None,None,None,None,None
691643,2023-11-30 17:59:59,30,3,17:59:59,17,0,1,50.7,59.400002,32.700001,...,17:52:14,8,14.636209,121.080691,None,None,None,None,None,None


# Convert Datetime to Numeric Values

In [231]:
# Convert all the times into a timestamp with 0 = 00:00:00 and 86399 = 23:59:59
df['Datetime'] = df['Datetime'].apply(lambda time: time_to_daytimestamp(time))

df['Time'] = df['Time'].apply(lambda time: time_to_daytimestamp(time))

df['Previous Time to Xavier Hall'] = df['Previous Time to Xavier Hall'].apply(lambda time: time_to_daytimestamp(time))
df['Previous Time to Fine Arts Annex'] = df['Previous Time to Fine Arts Annex'].apply(lambda time: time_to_daytimestamp(time))
df['Previous Time to Loyola House of Studies'] = df['Previous Time to Loyola House of Studies'].apply(lambda time: time_to_daytimestamp(time))
df['Previous Time to Grade School'] = df['Previous Time to Grade School'].apply(lambda time: time_to_daytimestamp(time))
df['Previous Time to Gate 2.5'] = df['Previous Time to Gate 2.5'].apply(lambda time: time_to_daytimestamp(time))
df['Previous Time to Leong Hall'] = df['Previous Time to Leong Hall'].apply(lambda time: time_to_daytimestamp(time))

df['Next Time to Xavier Hall'] = df['Next Time to Xavier Hall'].apply(lambda time: time_to_daytimestamp(time))
df['Next Time to Fine Arts Annex'] = df['Next Time to Fine Arts Annex'].apply(lambda time: time_to_daytimestamp(time))
df['Next Time to Loyola House of Studies'] = df['Next Time to Loyola House of Studies'].apply(lambda time: time_to_daytimestamp(time))
df['Next Time to Grade School'] = df['Next Time to Grade School'].apply(lambda time: time_to_daytimestamp(time))
df['Next Time to Gate 2.5'] = df['Next Time to Gate 2.5'].apply(lambda time: time_to_daytimestamp(time))
df['Next Time to Leong Hall'] = df['Next Time to Leong Hall'].apply(lambda time: time_to_daytimestamp(time))

df

,Datetime,Day,Encoded Day,Time,Hour_of_Day,IsEJeep1,IsEJeep2,Altitude,Humidity,Temperature,...,Previous Time to Leong Hall,Encoded Next Station,Next Station Lat,Next Station Long,Next Time to Xavier Hall,Next Time to Fine Arts Annex,Next Time to Loyola House of Studies,Next Time to Grade School,Next Time to Gate 2.5,Next Time to Leong Hall
388827,63826639200,9,3,21600,6,1,0,50.3,77.699997,29.500000,...,-1,1,14.634715,121.076248,23020,24489,23214,22598,23751,22849
388828,63826639201,9,3,21601,6,1,0,50.1,77.699997,29.500000,...,-1,1,14.634715,121.076248,23020,24489,23214,22598,23751,22849
388829,63826639202,9,3,21602,6,1,0,50.1,77.699997,29.500000,...,-1,1,14.634715,121.076248,23020,24489,23214,22598,23751,22849
388830,63826639203,9,3,21603,6,1,0,49.8,77.699997,29.500000,...,-1,1,14.634715,121.076248,23020,24489,23214,22598,23751,22849
388831,63826639204,9,3,21604,6,1,0,49.8,77.699997,29.500000,...,-1,1,14.634715,121.076248,23020,24489,23214,22598,23751,22849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691640,63828496796,30,3,64796,17,0,1,53.1,59.400002,32.700001,...,64334,8,14.636209,121.080691,-1,-1,-1,-1,-1,-1
691641,63828496797,30,3,64797,17,0,1,52.6,59.400002,32.700001,...,64334,8,14.636209,121.080691,-1,-1,-1,-1,-1,-1
691642,63828496798,30,3,64798,17,0,1,51.8,59.400002,32.700001,...,64334,8,14.636209,121.080691,-1,-1,-1,-1,-1,-1
691643,63828496799,30,3,64799,17,0,1,50.7,59.400002,32.700001,...,64334,8,14.636209,121.080691,-1,-1,-1,-1,-1,-1


In [232]:
# make sure there are no nan values
df = df.fillna(-1)

In [233]:
# Sort by datetime
df = df.sort_values(by=['Day', 'IsEJeep2', 'Time'], ascending=True)

In [234]:
test_df = df
test_df.reset_index(drop=True, inplace=True)

# Scalling

In [235]:
# # split data into train-test set
# train_df = df[df['Day'] < 29]
# test_df = df[df['Day'] >= 29]
# validation_df = df[df['Day'] < 30]

In [236]:
from sklearn.preprocessing import MinMaxScaler

# Extract the numerical columns for scaling
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler on the training data and transform both the training and test data
# train_df[numerical_columns] = scaler.fit_transform(train_df[numerical_columns])
# test_df[numerical_columns] = scaler.transform(test_df[numerical_columns])
# validation_df[numerical_columns] = scaler.transform(validation_df[numerical_columns])
# train_df

# Fit the scaler on entire data
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])
print(f"colNames = {list(df.columns)}")

colNames = ['Datetime', 'Day', 'Encoded Day', 'Time', 'Hour_of_Day', 'IsEJeep1', 'IsEJeep2', 'Altitude', 'Humidity', 'Temperature', 'Latitude', 'Longitude', 'Lat Diff', 'Long Diff', 'Distance', 'Cum Distance', 'Abs Distance', 'Percent Distance', 'Encoded Station', 'IsStation', 'IsCharging', 'Encoded Prev Station', 'Prev Station Lat', 'Prev Station Long', 'Prev Abs Distance', 'Prev Cum Distance', 'Previous Time to Xavier Hall', 'Previous Time to Fine Arts Annex', 'Previous Time to Loyola House of Studies', 'Previous Time to Grade School', 'Previous Time to Gate 2.5', 'Previous Time to Leong Hall', 'Encoded Next Station', 'Next Station Lat', 'Next Station Long', 'Next Time to Xavier Hall', 'Next Time to Fine Arts Annex', 'Next Time to Loyola House of Studies', 'Next Time to Grade School', 'Next Time to Gate 2.5', 'Next Time to Leong Hall']


In [237]:
import joblib
scaler_filename = "scaler.save"
joblib.dump(scaler, scaler_filename) 

['scaler.save']

In [238]:
# copy this to unscale

scaler = joblib.load("scaler.save")
def invTransform(scaler, data, colName, colNames):
    dummy = pd.DataFrame(np.zeros((len(data), len(colNames))), columns=colNames)
    dummy[colName] = data
    dummy = pd.DataFrame(scaler.inverse_transform(dummy), columns=colNames)
    return dummy[colName].values

# test['Next Station Long'] = invTransform(scaler, test['Next Station Long'], 'Next Station Long', df.columns)

## Exporting data

In [ ]:
df.to_csv('Out_v4.csv')
# train_df.to_csv('Train_Out.csv')
# test_df.to_csv('Test_Out.csv')
# validation_df.to_csv('Validation_Out.csv')

# To import this data


In [239]:
# import data boilerplate

df = pd.read_csv('Out_v4.csv')

df = df.drop(columns=['Unnamed: 0'])

train = df[df['Day'] < 29]
test = df[df['Day'] >= 29]

y_columns = ['Next Time to Xavier Hall', 'Next Time to Fine Arts Annex', 'Next Time to Loyola House of Studies', 'Next Time to Grade School', 'Next Time to Gate 2.5', 'Next Time to Leong Hall']

train_X = train.drop(columns=y_columns + ['Datetime'])
train_Y = train[y_columns]

test_X = test.drop(columns=y_columns + ['Datetime'])
test_Y = test[y_columns]

In [242]:
import joblib

# copy this to unscale
scaler = joblib.load("scaler.save")
def unscale(scaler, data, colName, colNames):
    dummy = pd.DataFrame(np.zeros((len(data), len(colNames))), columns=colNames)
    dummy[colName] = data
    dummy = pd.DataFrame(scaler.inverse_transform(dummy), columns=colNames)
    return dummy[colName].values


# example use case
test = pd.DataFrame(columns=['Altitude', 'Next Station Lat', 'Next Time to Gate 2.5'])
test.loc[0] = [0.5, 0.9, 0.3]

print("Scaled data:", test)

test['Altitude'] = unscale(scaler, test['Altitude'], 'Altitude', df.columns)

test['Next Station Lat'] = unscale(scaler, test['Next Station Lat'], 'Next Station Lat', df.columns)

test['Next Time to Gate 2.5'] = unscale(scaler, test['Next Time to Gate 2.5'], 'Next Time to Gate 2.5', df.columns)

print("Unscaled data:", test)


Scaled data:    Altitude  Next Station Lat  Next Time to Gate 2.5
0       0.5               0.9                    0.3
Unscaled data:    Altitude  Next Station Lat  Next Time to Gate 2.5
0    153.45         14.640186                19251.8


In [ ]:
scaler = joblib.load("scaler.save")
print(scaler)

# Plot all columns

In [ ]:
# plot each column of df
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
plt.figure(figsize=(20, 80))
for i, col in enumerate(df.columns):
    plt.subplot(len(df.columns), 1, i+1)
    plt.plot(df[col])
    plt.title(col, y=0.5, loc='right') 
plt.show()


# Final_Out reader

In [ ]:
# final_out = pd.read_csv('../Finalized Models/Final_Out.csv',)
# final_out

# Raw Data Observer

In [ ]:

# ejeep1 = process_file("E-Jeep Data/ejeep1/data_2023-11-30_1.csv")
# ejeep2 = process_file("E-Jeep Data/ejeep2/data_2023-11-30_2.csv")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

step = 0
duration = 12

raw = ejeep1[60*60*(step):60*60*(step+duration)]
raw2 = ejeep2[60*60*(step):60*60*(step+duration)]

# Plot latitude and longitude over time on a 2x3 grid
plt.figure(figsize=(10, 12))

# Plot latitude over time
plt.subplot(3, 2, 1)
plt.plot(raw['Datetime'], raw['Latitude'])
plt.xlabel('Time')
plt.ylabel('Latitude')
plt.title('Latitude over Time')

# Plot longitude over time
plt.subplot(3, 2, 3)
plt.plot(raw['Datetime'], raw['Longitude'])
plt.xlabel('Time')
plt.ylabel('Longitude')
plt.title('Longitude over Time')

# Plot latitude over time for raw2
plt.subplot(3, 2, 2)
plt.plot(raw2['Datetime'], raw2['Latitude'])
plt.xlabel('Time')
plt.ylabel('Latitude')
plt.title('Latitude over Time (raw2)')

# Plot longitude over time for raw2
plt.subplot(3, 2, 4)
plt.plot(raw2['Datetime'], raw2['Longitude'])
plt.xlabel('Time')
plt.ylabel('Longitude')
plt.title('Longitude over Time (raw2)')

# Plot latitude vs longitude
plt.subplot(3, 2, 5)
plt.scatter(raw['Longitude'], raw['Latitude'], s=0.3)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Latitude vs Longitude')

# Plot latitude vs longitude for raw2
plt.subplot(3, 2, 6)
plt.scatter(raw2['Longitude'], raw2['Latitude'], s=0.3)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Latitude vs Longitude (raw2)')

plt.tight_layout()
plt.show()



In [ ]:
test = pd.read_csv("E-Jeep Data/ejeep1/data_2023-11-14_1.csv")
test